# Treinamento, teste e avaliação com os imoveis filtrados a venda

## Importação das Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import locale

# Ferramentas do Scikit-learn
from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    cross_val_score
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score

# Modelos a serem comparados
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

## Configurações de Visualização

In [2]:
# Configura o locale para formatar valores em Reais (BRL)
try:
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
    pd.options.display.float_format = 'R$ {:,.2f}'.format
except:
    print("Locale 'pt_BR.UTF-8' não encontrado. Usando locale padrão.")
    pd.options.display.float_format = '{:,.2f}'.format

# Configurações do Seaborn para os gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## Carregar e Preparar os Dados

In [3]:
# Carregar o dataset de venda
try:
    df_venda = pd.read_csv('../Dados/imoveis_locacao_filtrado.csv')
    print("Dados de venda carregados com sucesso!")
except FileNotFoundError:
    print("Erro: Arquivo 'imoveis_locacao_filtrado.csv' não encontrado.")

print("\nVisão inicial dos dados:")
print(df_venda.head())
print("\nTipos de dados:")
df_venda.info()

Dados de venda carregados com sucesso!

Visão inicial dos dados:
          tipo                  bairro     cidade objetivo  area_util  \
0         casa          americanopolis  são paulo  Locação   R$ 40.00   
1         casa         vila_gumercindo  são paulo  Locação   R$ 25.00   
2         casa         vila_gumercindo  são paulo  Locação   R$ 25.00   
3         casa            vila_fachini  são paulo  Locação   R$ 50.00   
4  apartamento  jardim_miriam_zona_sul  são paulo  Locação   R$ 30.00   

   quartos  suites  vagas     preco  
0        1       0      0 R$ 800.00  
1        1       0      0 R$ 850.00  
2        1       0      0 R$ 850.00  
3        1       0      0 R$ 850.00  
4        1       0      0 R$ 850.00  

Tipos de dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tipo       853 non-null    object 
 1   bairro     853 non-nul

## Engenharia de Features

In [4]:
print("\nIniciando Engenharia de Features...")

# 1. Criar features binárias simples
df_venda['tem_suite'] = (df_venda['suites'] > 0).astype(int)
df_venda['tem_vaga'] = (df_venda['vagas'] > 0).astype(int)

print("Novas features criadas: 'tem_suite', 'tem_vaga'")


Iniciando Engenharia de Features...
Novas features criadas: 'tem_suite', 'tem_vaga'


## Definição de Features (X) e Alvo (y)

In [5]:
# --- Variável Alvo (y) ---

# Aplicar a transformação logarítmica no PRECO. Isso estabiliza a variância e melhora a performance de todos os modelos.
y = np.log1p(df_venda['preco'])

print(f"\nVariável alvo 'y' criada com np.log1p(preco). Média (log): {y.mean():.4f}")

# --- Features (X) ---

# Definir quais colunas serão usadas para prever o preço
features_numericas = [
    'area_util', 
    'quartos', 
    'suites', 
    'vagas',
    # Novas features:
    'tem_suite',
    'tem_vaga'
]

features_categoricas = [
    'tipo',
    'bairro'
]

X = df_venda[features_numericas + features_categoricas]

print("Features (X) e Alvo (y) definidos.")
print(f"Total de features em X: {len(features_numericas)} numéricas e {len(features_categoricas)} categóricas.")


Variável alvo 'y' criada com np.log1p(preco). Média (log): 8.3232
Features (X) e Alvo (y) definidos.
Total de features em X: 6 numéricas e 2 categóricas.


## Divisão em Dados de Treino e Teste

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Dados divididos: {len(X_train)} para treino, {len(X_test)} para teste.")

Dados divididos: 682 para treino, 171 para teste.


## Construção do Pipeline de Pré-processamento

In [7]:
# 1. Pipeline para features numéricas:
#    - StandardScaler: Coloca todas as features na mesma escala (média 0, desvio 1)
# Isso é crucial para modelos como Regressão Linear e evita que features com valores grandes (como 'area_util') dominem as com valores pequenos (como 'quartos').
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# 2. Pipeline para features categóricas:
#    - OneHotEncoder: Transforma 'bairro' e 'tipo' em colunas binárias (0 ou 1)
#    - handle_unknown='ignore': Se um bairro novo aparecer nos dados de teste, ele será ignorado sem quebrar o modelo.
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 3. Combinar os pipelines com o ColumnTransformer
#    Ele aplicará o transformador correto para a coluna correta.
preprocessor = ColumnTransformer(
    transformers=[
        # ('num', ...): Aplica o 'numeric_transformer' na lista de colunas 'features_numericas'.
        ('num', numeric_transformer, features_numericas),

        # ('cat', ...): Aplica o 'categorical_transformer' na lista 'features_categoricas'.
        ('cat', categorical_transformer, features_categoricas)
    ],
    # remainder='passthrough': Define o que fazer com colunas que NÃO foram listadas em 'features_numericas' ou 'features_categoricas' 'passthrough' significa "mantenha-as como estão".
    remainder='passthrough'
)

print("\nPipeline de pré-processamento (ColumnTransformer) construído.")


Pipeline de pré-processamento (ColumnTransformer) construído.


## Criação dos Pipelines de Modelo Completos

In [8]:
# --- Modelo 1: Regressão Linear ---
# Cria um "Pipeline" (fluxo de trabalho) para o modelo.
pipeline_lr = Pipeline(steps=[
    # Passo 1: Aplicar o 'preprocessor' (StandardScaler + OneHotEncoder) que definimos antes.
    ('preprocessor', preprocessor),
    # Passo 2: Após pré-processar, treinar o modelo de Regressão Linear.
    ('model', LinearRegression())
])

# --- Modelo 2: Random Forest (com parâmetros iniciais) ---
# Cria um segundo pipeline mas com um modelo diferente.
pipeline_rf = Pipeline(steps=[
    # Passo 1: Aplicar o mesmo pré-processador.
    ('preprocessor', preprocessor),
    # Passo 2: Treinar o modelo Random Forest.
    ('model', RandomForestRegressor(
        n_estimators=100, # Define que o modelo usará 100 árvores.
        random_state=42,  # Garante que o resultado seja o mesmo toda vez que rodar.
        n_jobs=-1         # Usa todos os núcleos do processador para treinar mais rápido.
    ))
])

# --- Modelo 3: XGBoost (com parâmetros iniciais) ---
# Cria o terceiro pipeline com o XGBoost
pipeline_xgb = Pipeline(steps=[
    # Passo 1: Aplicar o mesmo pré-processador.
    ('preprocessor', preprocessor),
    # Passo 2: Treinar o modelo XGBoost.
    ('model', XGBRegressor(
        random_state=42, # Garante resultados reprodutíveis.
        n_jobs=-1        # Usa todos os núcleos para acelerar.
    ))
])

# Cria um dicionário chamado 'models' para organizar os pipelines.
# A chave é um nome, e o valor é o objeto do pipeline.
models = {
    'Regressão Linear': pipeline_lr,
    'Random Forest': pipeline_rf,
    'XGBoost': pipeline_xgb
}

print("Pipelines completos (Pré-processamento + Modelo) criados.")

Pipelines completos (Pré-processamento + Modelo) criados.


## Avaliação Base (Cross-Validation)

In [9]:
print("\n--- Iniciando Validação Cruzada (Baseline) ---")
print("Avaliando modelos nos dados de treino (CV=5)...")

# Nota: Usamos 'neg_mean_absolute_error' (Erro Médio Absoluto Negativo) porque a função cross_val_score sempre tenta maximizar uma pontuação. Como queremos minimizar o erro (MAE), usamos a versão negativa dele. O MAE real será o valor *negativo* disso, na escala LOG.

for name, model in models.items():

    # Esta é a função principal: ela executa a Validação Cruzada. Ela pega o 'model', divide 'X_train' e 'y_train' em 5 partes ('cv=5'). Treina em 4 partes e testa em 1, repetindo isso 5 vezes. 'scoring' define a métrica (MAE negativo). 'n_jobs=-1' usa todos os núcleos do processador para rodar mais rápido. 'scores' será uma lista com 5 pontuações (uma para cada fold).
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    
    # Calcula a média das 5 pontuações (ex: -0.21, -0.22, -0.20...). Invertemos o sinal (multiplicamos por -1) para ter o MAE positivo (ex: 0.21). Este é o erro médio do modelo na escala LOG.
    mae_log_mean = -np.mean(scores)
    mae_log_std = np.std(scores)
    
    print(f"Modelo: {name}")
    print(f"  MAE Médio (log): {mae_log_mean:.4f} (std: +/- {mae_log_std:.4f})")

print("--- Validação Cruzada Concluída ---")


--- Iniciando Validação Cruzada (Baseline) ---
Avaliando modelos nos dados de treino (CV=5)...
Modelo: Regressão Linear
  MAE Médio (log): 0.2811 (std: +/- 0.0280)
Modelo: Random Forest
  MAE Médio (log): 0.2805 (std: +/- 0.0137)
Modelo: XGBoost
  MAE Médio (log): 0.2808 (std: +/- 0.0110)
--- Validação Cruzada Concluída ---


## Avaliação Final no Conjunto de Teste

In [10]:
print("\n--- Avaliação Final no Conjunto de Teste ---")

# Treinar os modelos base
if 'Regressão Linear' in models:
    models['Regressão Linear'].fit(X_train, y_train)
if 'Random Forest' in models:
    models['Random Forest'].fit(X_train, y_train)
if 'XGBoost' in models:
    models['XGBoost'].fit(X_train, y_train)

print("Modelos base treinados.")

# Cria um dicionário vazio para guardar as previsões
predictions = {}
# Cria uma lista vazia para guardar as métricas (MAE, R²)
results_list = []

# Converter y_test (log) de volta para a escala original (R$)
y_test_original = np.expm1(y_test)

for name, model in models.items():
    
    # 1. Fazer previsão na escala LOG
    y_pred_log = model.predict(X_test)
    
    # 2. Converter previsão de volta para a escala ORIGINAL (R$)
    y_pred_original = np.expm1(y_pred_log)
    
    # Salvar previsões originais
    predictions[name] = y_pred_original
    
    # 3. Calcula as métricas de performance, comparando o Real vs. Previsto.
    
    # MAE (Erro Médio Absoluto): A métrica principal. Diz, em média, "quantos Reais" o modelo errou (para mais ou para menos).
    mae = mean_absolute_error(y_test_original, y_pred_original)

    # R² (R-quadrado): A métrica de "qualidade do ajuste". Diz qual porcentagem da variação dos preços o modelo conseguiu explicar (0 a 1).
    r2 = r2_score(y_test_original, y_pred_original)
    
    # Adiciona os resultados deste modelo na lista.
    results_list.append({'Modelo': name, 'MAE (R$)': mae, 'R²': r2})

# Transforma a lista de resultados em uma tabela (DataFrame) do Pandas.
# Ordena a tabela pelo 'MAE (R$)' (do menor para o maior) para que o melhor modelo apareça primeiro.
df_results = pd.DataFrame(results_list).sort_values(by='MAE (R$)')

print("\nResultados da Avaliação Final (em R$):")

# Imprime a tabela de resultados de forma legível:
# .to_string() -> Converte o DataFrame para texto.
# formatters={...} -> Define a formatação (MAE como "R$ 123.456,78" e R² como "0.745").
# index=False -> Esconde os números de índice (0, 1, 2...) do DataFrame.
print(df_results.to_string(formatters={'MAE (R$)': 'R$ {:,.2f}'.format, 'R²': '{:.3f}'.format}, index=False))


--- Avaliação Final no Conjunto de Teste ---
Modelos base treinados.

Resultados da Avaliação Final (em R$):
          Modelo    MAE (R$)    R²
Regressão Linear R$ 1,548.14 0.666
   Random Forest R$ 1,662.11 0.430
         XGBoost R$ 1,770.38 0.410


In [11]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV

## Otimização da Regressão Linear

In [15]:
print("\n--- Iniciando Otimização da Regressão Linear ---")

# 1. CRIAR UM NOVO PRÉ-PROCESSADOR COM FEATURES POLINOMIAIS

# O objetivo é capturar relações não-lineares (ex: area², area * quartos)

# 1a. Novo Pipeline Numérico (com PolynomialFeatures + Scaler)
numeric_transformer_poly = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
])

# 1b. Pipeline Categórico (igual ao anterior)
categorical_transformer_poly = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 1c. Novo ColumnTransformer (usa os pipelines acima)
preprocessor_poly = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_poly, features_numericas),
        ('cat', categorical_transformer_poly, features_categoricas)
    ],
    remainder='passthrough'
)

print("Novo pré-processador com PolynomialFeatures criado.")

# 2. DEFINIR OS NOVOS MODELOS LINEARES OTIMIZADOS

# Lista de 'alphas' (força da penalidade) para os modelos testarem
alphas_para_testar = [0.1, 0.5, 1.0, 5.0, 10.0, 20.0, 50.0]

# 2a. Modelo Ridge (L2): Encolhe os coeficientes
pipeline_ridge = Pipeline(steps=[
    ('preprocessor', preprocessor_poly),
    ('model', RidgeCV(alphas=alphas_para_testar))
])

# 2b. Modelo Lasso (L1): Pode zerar coeficientes inúteis
pipeline_lasso = Pipeline(steps=[
    ('preprocessor', preprocessor_poly),
    ('model', LassoCV(alphas=alphas_para_testar, cv=5, max_iter=5000)) # Lasso pode precisar de mais iterações
])

# 2c. Dicionário com os novos modelos
models_otimizados = {
    'Regressão Linear (Otimizada com Ridge)': pipeline_ridge,
    'Regressão Linear (Otimizada com Lasso)': pipeline_lasso
}

print("Novos modelos (Ridge e Lasso) criados.")


--- Iniciando Otimização da Regressão Linear ---
Novo pré-processador com PolynomialFeatures criado.
Novos modelos (Ridge e Lasso) criados.


In [16]:
# TREINAR E AVALIAR OS NOVOS MODELOS

print("\n--- Iniciando Avaliação dos Modelos Otimizados ---")

results_list_otimizada = []

for name, model in models_otimizados.items():
    
    print(f"Treinando e avaliando: {name}...")
    
    # Treina o modelo
    model.fit(X_train, y_train)
    
    # Faz a previsão (em log)
    y_pred_log = model.predict(X_test)
    
    # Converte de volta para R$
    y_pred_original = np.expm1(y_pred_log)
    
    # Salva as previsões
    predictions[name] = y_pred_original
    
    # Calcula as métricas
    mae = mean_absolute_error(y_test_original, y_pred_original)
    r2 = r2_score(y_test_original, y_pred_original)
    
    results_list_otimizada.append({'Modelo': name, 'MAE (R$)': mae, 'R²': r2})


# 4. EXIBIR OS RESULTADOS FINAIS DA OTIMIZAÇÃO

df_results_otimizada = pd.DataFrame(results_list_otimizada).sort_values(by='MAE (R$)')

print("\n--- Resultados da Avaliação OTIMIZADA (em R$) ---")
print(df_results_otimizada.to_string(formatters={'MAE (R$)': 'R$ {:,.2f}'.format, 'R²': '{:.3f}'.format}, index=False))

print("\n--- Para Comparação (Resultados da Célula 11) ---")
print(df_results.to_string(formatters={'MAE (R$)': 'R$ {:,.2f}'.format, 'R²': '{:.3f}'.format}, index=False))


--- Iniciando Avaliação dos Modelos Otimizados ---
Treinando e avaliando: Regressão Linear (Otimizada com Ridge)...
Treinando e avaliando: Regressão Linear (Otimizada com Lasso)...

--- Resultados da Avaliação OTIMIZADA (em R$) ---
                                Modelo    MAE (R$)    R²
Regressão Linear (Otimizada com Ridge) R$ 1,523.06 0.596
Regressão Linear (Otimizada com Lasso) R$ 2,015.27 0.307

--- Para Comparação (Resultados da Célula 11) ---
          Modelo    MAE (R$)    R²
Regressão Linear R$ 1,548.14 0.666
   Random Forest R$ 1,662.11 0.430
         XGBoost R$ 1,770.38 0.410


In [19]:
print("\n--- Iniciando Otimização V2 (Polynomial Seletivo) ---")

# 1. DEFINIR AS FEATURES NUMÉRICAS

# Feature que receberá PolynomialFeatures (relação não-linear)
features_poly = ['area_util']

# Features que NÃO receberão PolynomialFeatures (relação linear)
features_linear = [
    'quartos', 
    'suites', 
    'vagas',
    'tem_suite',
    'tem_vaga',
    'area_por_quarto', 
    'proporcao_suites'
]

# Garante que as listas estão corretas e não contêm features que não existem
features_poly = [f for f in features_poly if f in X_train.columns]
features_linear = [f for f in features_linear if f in X_train.columns]


# 2. CRIAR UM PRÉ-PROCESSADOR MAIS INTELIGENTE

# 2a. Pipeline para 'area_util' (Polynomial + Scaler)
numeric_transformer_poly = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
])

# 2b. Pipeline para as outras features numéricas (Apenas Scaler)
numeric_transformer_linear = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# 2c. Pipeline Categórico (igual ao anterior)
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 2d. Novo ColumnTransformer (que junta os 3 pipelines)
preprocessor_v2 = ColumnTransformer(
    transformers=[
        ('poly', numeric_transformer_poly, features_poly), # Pipeline 1
        ('linear', numeric_transformer_linear, features_linear), # Pipeline 2
        ('cat', categorical_transformer, features_categoricas) # Pipeline 3
    ],
    remainder='passthrough'
)

print("Novo pré-processador (V2) seletivo criado.")

# 3. DEFINIR O MODELO RIDGE COM ALPHAS REFINADOS

# Alphas mais "finos" e focados em valores menores
alphas_refinados = [0.01, 0.05, 0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0]

pipeline_ridge_v2 = Pipeline(steps=[
    ('preprocessor', preprocessor_v2), # Usa o novo pré-processador V2
    ('model', RidgeCV(alphas=alphas_refinados))
])

models_otimizados_v2 = {
    'Regressão Linear (Otimizada V2 - Ridge)': pipeline_ridge_v2
}

print("Novos modelos (Ridge V2) criados.")

# 4. TREINAR E AVALIAR O NOVO MODELO

print("\n--- Iniciando Avaliação do Modelo Otimizado V2 ---")

results_list_v2 = []

for name, model in models_otimizados_v2.items():
    
    print(f"Treinando e avaliando: {name}...")
    
    model.fit(X_train, y_train)
    y_pred_log = model.predict(X_test)
    y_pred_original = np.expm1(y_pred_log)
    
    predictions[name] = y_pred_original
    
    mae = mean_absolute_error(y_test_original, y_pred_original)
    r2 = r2_score(y_test_original, y_pred_original)
    
    results_list_v2.append({'Modelo': name, 'MAE (R$)': mae, 'R²': r2})

# 5. EXIBIR OS RESULTADOS FINAIS (V2)

df_results_v2 = pd.DataFrame(results_list_v2).sort_values(by='MAE (R$)')

print("\n--- Resultados da Avaliação OTIMIZADA (V2) ---")
print(df_results_v2.to_string(formatters={'MAE (R$)': 'R$ {:,.2f}'.format, 'R²': '{:.3f}'.format}, index=False))

print("\n--- Para Comparação (Resultados Anteriores) ---")
print("Modelo Otimizado (V1):")
print(df_results_otimizada.to_string(formatters={'MAE (R$)': 'R$ {:,.2f}'.format, 'R²': '{:.3f}'.format}, index=False))
print("\nModelo Original (Célula 11):")
print(df_results.to_string(formatters={'MAE (R$)': 'R$ {:,.2f}'.format, 'R²': '{:.3f}'.format}, index=False))


--- Iniciando Otimização V2 (Polynomial Seletivo) ---
Novo pré-processador (V2) seletivo criado.
Novos modelos (Ridge V2) criados.

--- Iniciando Avaliação do Modelo Otimizado V2 ---
Treinando e avaliando: Regressão Linear (Otimizada V2 - Ridge)...

--- Resultados da Avaliação OTIMIZADA (V2) ---
                                 Modelo    MAE (R$)    R²
Regressão Linear (Otimizada V2 - Ridge) R$ 1,473.18 0.596

--- Para Comparação (Resultados Anteriores) ---
Modelo Otimizado (V1):
                                Modelo    MAE (R$)    R²
Regressão Linear (Otimizada com Ridge) R$ 1,523.06 0.596
Regressão Linear (Otimizada com Lasso) R$ 2,015.27 0.307

Modelo Original (Célula 11):
          Modelo    MAE (R$)    R²
Regressão Linear R$ 1,548.14 0.666
   Random Forest R$ 1,662.11 0.430
         XGBoost R$ 1,770.38 0.410


## Análise de Coeficientes do Modelo Otimizado

### Modelo V1

In [21]:
print("\n--- Análise de Coeficientes (Regressão Linear Otimizada) ---")

# Escolha o nome do seu melhor modelo da célula anterior
nome_do_melhor_modelo = df_results_otimizada.iloc[0]['Modelo'] 

print(f"Analisando o modelo: {nome_do_melhor_modelo}")

try:
    # 1. Pega o pipeline treinado
    pipeline_treinado = models_otimizados[nome_do_melhor_modelo]
    
    # 2. Pega o modelo (RidgeCV ou LassoCV) de dentro do pipeline
    modelo_final = pipeline_treinado.named_steps['model']

    # 3. Pega o pré-processador de dentro do pipeline
    preprocessor_fitted = pipeline_treinado.named_steps['preprocessor']

    # --- Pegar todos os nomes das features (isto é complexo) ---
    # 3a. Nomes das features numéricas (criadas pelo PolynomialFeatures)
    num_features_poly = (
        preprocessor_fitted.named_transformers_['num'] 
        .named_steps['poly']
        .get_feature_names_out(features_numericas)
    )
    
    # 3b. Nomes das features categóricas (criadas pelo OneHotEncoder)
    cat_features_out = (
        preprocessor_fitted.named_transformers_['cat']
        .named_steps['onehot']
        .get_feature_names_out(features_categoricas)
    )
    
    # 3c. Lista final com TODOS os nomes
    all_features_out = list(num_features_poly) + list(cat_features_out)

    # 4. Pega os coeficientes (os "pesos" que o modelo deu a cada feature)
    coefs = modelo_final.coef_

    # 5. Cria o DataFrame de coeficientes
    df_coefs = pd.DataFrame({
        'Feature': all_features_out,
        'Coeficiente (em log)': coefs
    }).sort_values(by='Coeficiente (em log)', ascending=False)
    
    # 6. Exibe os resultados
    print("\nAlpha (penalidade) escolhido pelo modelo:", modelo_final.alpha_)

    print("\n--- Top 15 Features que MAIS AUMENTAM o aluguel ---")
    df_coefs_head = df_coefs[df_coefs['Feature'].str.len() < 40].head(15)
    print(df_coefs_head.to_string(formatters={'Coeficiente (em log)': '{:,.4f}'.format}))

    print("\n--- Top 15 Features que MAIS DIMINUEM o aluguel ---")
    df_coefs_tail = df_coefs[df_coefs['Feature'].str.len() < 40].tail(15)
    print(df_coefs_tail.to_string(formatters={'Coeficiente (em log)': '{:,.4f}'.format}))

except Exception as e:
    print(f"Não foi possível extrair coeficientes: {e}")


--- Análise de Coeficientes (Regressão Linear Otimizada) ---
Analisando o modelo: Regressão Linear (Otimizada com Ridge)

Alpha (penalidade) escolhido pelo modelo: 0.5

--- Top 15 Features que MAIS AUMENTAM o aluguel ---
                        Feature Coeficiente (em log)
53          bairro_higienopolis               0.6701
130  bairro_vila_nova_conceicao               0.6650
67         bairro_jardim_europa               0.6599
0                     area_util               0.6517
36         bairro_brooklin_novo               0.6290
35              bairro_brooklin               0.6042
40       bairro_cerqueira_cesar               0.5980
56            bairro_itaim_bibi               0.5956
47        bairro_cidade_moncoes               0.5947
93           bairro_real_parque               0.5598
131         bairro_vila_olimpia               0.5447
73     bairro_jardim_paulistano               0.5376
91             bairro_pinheiros               0.4822
72       bairro_jardim_paulista     

### Modelo V2

In [22]:
print("\n--- Análise de Coeficientes (Regressão Linear Otimizada V2) ---")

# Vamos usar os resultados do DataFrame V2
nome_do_melhor_modelo = df_results_v2.iloc[0]['Modelo'] 

print(f"Analisando o modelo: {nome_do_melhor_modelo}")

try:
    # 1. Pega o pipeline treinado (do dicionário V2)
    pipeline_treinado = models_otimizados_v2[nome_do_melhor_modelo]
    
    # 2. Pega o modelo (RidgeCV) de dentro do pipeline
    modelo_final = pipeline_treinado.named_steps['model']

    # 3. Pega o pré-processador (V2) de dentro do pipeline
    preprocessor_fitted = pipeline_treinado.named_steps['preprocessor']

    # --- Pegar todos os nomes das features ---
    
    # 3a. Nomes das features POLINOMIAIS (do pipeline 'poly')
    poly_feature_names = (
        preprocessor_fitted.named_transformers_['poly'] # Pega o pipeline 'poly'
        .named_steps['poly'] # Pega o passo 'poly'
        .get_feature_names_out(features_poly) # Pega os nomes (usando 'features_poly')
    )
    
    # 3b. Nomes das features NUMÉRICAS LINEARES (do pipeline 'linear')
    #    (O StandardScaler não muda os nomes, então usamos a lista original)
    linear_feature_names = features_linear
    
    # 3c. Nomes das features CATEGÓRICAS (do pipeline 'cat')
    cat_features_out = (
        preprocessor_fitted.named_transformers_['cat'] # Pega o pipeline 'cat'
        .named_steps['onehot'] # Pega o passo 'onehot'
        .get_feature_names_out(features_categoricas) # Pega os nomes
    )
    
    # 3d. Lista final com TODOS os nomes (na ordem correta: poly, linear, cat)
    all_features_out = list(poly_feature_names) + list(linear_feature_names) + list(cat_features_out)

    # 4. Pega os coeficientes 
    coefs = modelo_final.coef_

    # 5. Cria o DataFrame de coeficientes
    df_coefs = pd.DataFrame({
        'Feature': all_features_out,
        'Coeficiente (em log)': coefs
    }).sort_values(by='Coeficiente (em log)', ascending=False)
    
    # 6. Exibe os resultados
    print("\nAlpha (penalidade) escolhido pelo modelo:", modelo_final.alpha_)

    print("\n--- Top 15 Features que MAIS AUMENTAM o aluguel ---")
    df_coefs_head = df_coefs[df_coefs['Feature'].str.len() < 40].head(15)
    print(df_coefs_head.to_string(formatters={'Coeficiente (em log)': '{:,.4f}'.format}))

    print("\n--- Top 15 Features que MAIS DIMINUEM o aluguel ---")
    df_coefs_tail = df_coefs[df_coefs['Feature'].str.len() < 40].tail(15)
    print(df_coefs_tail.to_string(formatters={'Coeficiente (em log)': '{:,.4f}'.format}))

except Exception as e:
    print(f"Não foi possível extrair coeficientes: {e}")
    print("Verifique se as variáveis 'models_otimizados_v2', 'df_results_v2', 'features_poly', e 'features_linear' existem.")


--- Análise de Coeficientes (Regressão Linear Otimizada V2) ---
Analisando o modelo: Regressão Linear (Otimizada V2 - Ridge)

Alpha (penalidade) escolhido pelo modelo: 1.0

--- Top 15 Features que MAIS AUMENTAM o aluguel ---
                         Feature Coeficiente (em log)
110   bairro_vila_nova_conceicao               0.6378
0                      area_util               0.6266
65   bairro_paineiras_do_morumbi               0.5933
36             bairro_itaim_bibi               0.5879
27         bairro_cidade_moncoes               0.5423
47          bairro_jardim_europa               0.5382
20        bairro_cerqueira_cesar               0.5079
15               bairro_brooklin               0.5028
53      bairro_jardim_paulistano               0.4953
16          bairro_brooklin_novo               0.4829
33           bairro_higienopolis               0.4760
111          bairro_vila_olimpia               0.4578
73            bairro_real_parque               0.4550
17      bairro_bro